In [1]:
%load_ext autoreload
%autoreload 2
import sys, warnings, yaml, logging, pandas, time, numpy
sys.path.append("../src/") # go to parent dir
from main import prepare_data, evaluate_models
warnings.filterwarnings('ignore')
with open('../conf/log.yml', 'rt') as f:
    config = yaml.safe_load(f.read())
logging.config.dictConfig(config)

In [ ]:
%load_ext autoreload
%autoreload 2
x_columns = ['train', 'gare', 'date', 'arret', 'p2q0', 'p3q0', 'p4q0', 'p0q2', 'p0q3', 'p0q4']
x_columns = ['gare', 'arret', 'p2q0', 'p3q0', 'p4q0', 'p0q2', 'p0q3', 'p0q4']
x_train, x_test, y_train, y_test, x_valid = prepare_data(x_columns)

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt

def plot_history(history):
    logging.info(history.history.keys())
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
def evaluate_model(model, plot=True):
    start = time.time()
    score = model.evaluate()
    end = time.time()
    score = model.evaluate()
    duration = numpy.round((end-start), 2)
    logging.info(f'{model.name}={numpy.round(score, 4)} in {duration}s')
    if plot:
        plot_history(model.history)
    return {'name': model.name, 'score': score, 'duration': duration}

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from models.factory import ModelFactory
with open('../conf/models.yml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, x_train, y_train, x_test, y_test)
lines = []
activations = ['celu', 'elu', 'gelu', 'glu', 'hard_sigmoid', 'hard_silu', 'hard_tanh', 'leaky_relu', 'linear', 'log_sigmoid', 'log_softmax', 'mish', 'relu', 
               'relu6', 'selu', 'sigmoid', 'silu', 'softmax', 'soft_shrink', 'softplus', 'softsign', 'sparse_plus', 'sparsemax', 'squareplus', 'tanh', 'tanh_shrink']

activations = ['softmax', 'tanh_shrink', 'elu', 'hard_silu', 'soft_shrink', 'log_softmax', 'relu', 'softplus']
activations = ['softmax', 'tanh_shrink', 'elu', 'hard_silu', 'soft_shrink', 'relu', 'softplus']
activations = ['elu']

optimizers = ['adam', 'sgd', 'rmsprop', 'adamw', 'adadelta', 'adagrad', 'adamax', 'adafactor', 'nadam', 'ftrl', 'lion', 'lamb']
# unitss = [5, 10, 15, 20, 25, 30]
unitss = [25]

optimizers = ['adam']
# nbr_layerss = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
nbr_layerss = [5] 

def  _evaluate_model(units, activation, optimizer, nbr_layers):
    model = factory.get_model('keras')
    model.params['layers'] = [{ 'units': units, 'activation': activation } for l in range(1, nbr_layers)]
    model.params['layers'].append({ 'units': 1, 'activation': 'linear' })
    activations_str = '_'.join([l['activation'] for l in model.params['layers']])
    model.name = f'{activations_str}_{optimizer}_{units}_{nbr_layers}'
    model.params['optimizer'] = optimizer
    line = evaluate_model(model, False)
    line['activation'] = activation
    line['optimizer'] = optimizer
    line['units'] = units
    return line

for activation in activations:
    for optimizer in optimizers:
        for units in unitss:
            for nbr_layers in nbr_layerss:
                line = _evaluate_model(units, activation, optimizer, nbr_layers)
                if not line is None:
                    lines.append(line)
df = pandas.DataFrame(lines).sort_values(by=['score'], ascending=True)
df

In [5]:
# df.to_csv('../data/keras_test_units.csv', index=False)